## LangGraph의 탄생배경

- RAG를 도입하면서 현실에서 다음과 같은 문제에 반복적으로 봉착한다.  
    - LLM이 생성한 답변이 실제 근거가 없는 hallucination인지 판단하기 어렵다.  
    - 문서 기반 검색(Retrieval)에서 얻은 정보가 질문의 맥락보다 외부 사전지식에 의존해 잘못된 답변을 만들 수 있다.  
    - 문서 검색으로 원하는 정보를 찾지 못하면 외부(웹, 논문 등)를 추가 탐색해야 하는데, 이 과정에서 신뢰할 수 없는 정보가 포함되면 최종 답변이 오염된다.  

- 가상의 예시: 기업 소개 문서로 QA를 수행할 때 매출액을 묻는 경우를 생각한다.  
    1. 기업 소개 문서에는 매출액 정보가 없다.  
    2. 시스템은 먼저 문서 검색을 수행하고 결과가 없으면 웹 검색으로 보강하려 시도한다.  
    3. 웹 검색에서 나온 정보가 출처가 불분명하거나 잘못된 경우, LLM은 그 정보를 근거로 답변을 만들어 hallucination을 발생시킬 수 있다.  
    4. 추가 검색-재검증 과정을 무한 반복하면 토큰 비용이 급증하고 응답 지연이 커진다.  

- 이처럼 단순한 action chain이 여러 도구와 의사결정 루프를 거치며 복잡해지면 파이프라인 유지와 디버깅이 어려워진다.  
    - 반복적 검색과 재작성 루프는 토큰 소모와 응답 지연을 키운다.  
    - 중간 결정(예: 결과 신뢰도 기준을 만족하지 않으면 검색을 중단) 로직이 늘어나면 파이프라인이 비선형적으로 복잡해진다.  
    - 각 단계의 작은 변화가 전체 답변 품질에 큰 영향을 끼쳐 예측 불가능성이 커진다.  

- Conventional RAG의 한계:  
    - 사전에 한정된 데이터 소스(PDF, DB, 테이블 등)에 의존한다.  
    - 고정된 청크 크기와 색인 전략을 사용한다.  
    - 쿼리 입력 형식과 검색 방법이 고정되어 유연성 부족하다.  
    - LLM/Agent의 불확실성으로 인해 신뢰성 확보가 어렵다.  
    - 고정된 프롬프트 템플릿으로 다양한 상황 대처가 어렵다.  
    - 이전 단계로 되돌아가 결과를 수정하거나 재검증하기 어렵다.  

## LangGraph 란?

- LangGraph는 RAG 파이프라인을 그래프 기반 워크플로우로 모델링하여 유연성과 투명성을 높이는 프레임워크이다.  
- LangChain처럼 도구 생태계를 활용할 수 있으면서도 LangChain에 종속되지 않는 경량 그래프 추상화를 제공한다.  
- 핵심 목표는 다음과 같다.  
    - 각 처리 단위를 독립적인 노드로 분리해 재사용성과 테스트 가능한 유닛으로 만든다.  
    - 노드 간의 흐름을 엣지로 정의해 조건부 분기와 반복, 병렬 처리를 자연스럽게 표현한다.  
    - 상태(state)를 명시적으로 관리해 중간 결과를 검증하거나 과거 실행을 재생(replay)할 수 있게 만든다.  

### 설계 아이디어와 장점  

- 모듈화: 데이터 적재, 전처리, 임베딩, 검색, 평가, 응답 생성 등 각 단계를 독립 노드로 구성해 조합 가능하게 한다.  
- 가시성: 그래프 구조로 흐름을 표현하면 어느 단계에서 문제가 발생했는지 추적하기 쉬워진다.  
- 유연성: 조건부 엣지로 다양한 분기 전략을 구현하고, human-in-the-loop을 손쉽게 끼워 넣을 수 있다.  
- 복구/재현성: 체크포인터를 통해 특정 시점으로 돌아가 중간 상태를 수정하고 재실행할 수 있다.  

### 예시 노드 목록  

 - **Checkpointer**: 과거 실행의 스냅샷을 저장하고 특정 시점으로 되돌려 재실행하거나 재검증하는 기능.  

## 구체적 가상 예시: 기업 매출 질문 시나리오  

- 상황: 내부 company_profile.pdf에 기업 개요와 연혁은 있지만 최신 매출액 정보는 없다.  
- 목표: 사용자 질문에 근거 기반 답변을 제공하되, 문서에 없을 경우 안전하게 외부 근거를 보강한다.  

- 권장 플로우(요약):  
  1. `document_loader` → `split` → `embedding` → `store`로 문서 색인 준비.  
  2. 사용자 질의 도착 → `query_rewrite`로 명확화(예: 기간, 통화 단위 추가).  
  3. `retriever`로 관련 청크 검색 → `relevance_evaluator`로 신뢰도 판단.  
  4. 관련도가 기준 이상이면 LLM 응답 생성 → `quality_evaluator`로 검증 후 반환.  
  5. 관련도가 낮거나 정보가 부족하면 `web_surfing`으로 외부 검색 시도 → `source_verifier`로 출처 신뢰성 점검.  
  6. 외부 출처가 신뢰할 수 있으면 LLM이 출처를 명시해 답변 생성, 그렇지 않으면 사용자에게 불확실성을 알리고 human-in-the-loop로 판단 요청.  

### 세부 동작과 안전장치  

- 신뢰도 기준: 검색 점수와 출처 메타데이터(도메인, 발행일, 공식성)를 결합해 임계값을 설정한다.  
- 토큰/비용 제어: 외부 검색은 최대 N번으로 제한하고, 재검색 루프는 체크포인터로 중단/재개할 수 있게 설계한다.  
- 출처 투명성: LLM 응답에 반드시 참조 출처를 병기하도록 강제한다.  
- Human fallback: 자동화가 불확실할 때는 검증 요청을 생성해 사람이 판단하도록 한다.  

### 간단한 pseudocode (개념 설명용)  

```python
# flow: document indexing
def index_documents(docs):  
    chunks = split(docs)  
    embeds = embed(chunks)  
    store(embeds)  

# flow: query handling
def handle_query(query):  
    query = query_rewrite(query)  
    results = retriever(query)  
    if relevance_evaluator(results):  
        answer = llm_generate(results, query)  
        if quality_evaluator(answer):  
            return answer_with_sources(answer, results)  
    external = web_surfing(query, max_attempts=2)  
    verified = source_verifier(external)  
    if verified:  
        return answer_with_sources(llm_generate(results+external, query), results+external)  
    return request_human_review(query, results, external)  
```

위 pseudocode는 개념을 설명하기 위한 간단한 예이며, 실제 구현에서는 비동기 처리, 에러 핸들링, 체크포인터 관리, 로깅 등이 필요하다.  

- **Node**: 특정 작업(task)을 수행하는 단위 함수 또는 컴포넌트.  
    - 입력과 출력을 명확히 정의하고 상태를 읽고 쓸 수 있다.  
- **Edge**: 노드 간의 실행 흐름을 정의하는 연결선.  
    - 단순 순차 실행뿐 아니라 조건부 분기, 반복, 병렬화를 표현한다.  
- **State**: 워크플로우 전반에서 공유되거나 노드별로 보존되는 값.  
    - 예: 쿼리 변형 히스토리, 검색 결과, LLM 응답, 평가 메타데이터 등.  
- **Conditional Edge**: 조건에 따라 다른 경로로 분기시키는 엣지.  
- **Human-in-the-loop**: 분기나 검증 단계에서 사람이 개입해 결정하는 지점.  
- **Checkpointer**: 과거 실행의 스냅샷을 저장하고 특정 시점으로 되돌려 재실행하거나 재검증하는 기능.  